In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# === 경로 설정 ===
base_dir = "/home/ricky/Data/Animal"
video_dir = os.path.join(base_dir, "video")
img_dir = os.path.join(base_dir, "images")
label_dir = os.path.join(base_dir, "labels")
mask_dir = os.path.join(base_dir, "masks")

os.makedirs(img_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# === 1단계: 영상 → 프레임 분할 ===
video_files = [f for f in os.listdir(video_dir) if f.endswith((".MP4", ".avi"))]

for video_file in video_files:
    video_path = os.path.join(video_dir, video_file)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"[오류] 영상 열기 실패: {video_path}")
        continue

    frame_idx = 0
    print(f"🎞️ 영상 분할 중: {video_file}")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = f"{os.path.splitext(video_file)[0]}_frame_{frame_idx:05d}.jpg"
        frame_path = os.path.join(img_dir, frame_name)
        cv2.imwrite(frame_path, frame)
        frame_idx += 1

    cap.release()
    print(f"✅ 프레임 저장 완료: {frame_idx}개 → {video_file}")

# === 2단계: 마스크 생성 ===
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith((".jpg", ".png"))])

for img_file in tqdm(img_files, desc="🖼️ 마스크 생성 중"):
    name = os.path.splitext(img_file)[0]
    img_path = os.path.join(img_dir, img_file)
    label_path = os.path.join(label_dir, name + ".txt")
    mask_path = os.path.join(mask_dir, name + ".png")

    img = cv2.imread(img_path)
    if img is None:                   
        print(f"[경고] 이미지 로드 실패: {img_path}")
        continue

    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)  # 기본은 검정 배경

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            if line.strip() == "":
                continue
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            _, x_center, y_center, box_w, box_h = map(float, parts)
            x1 = int((x_center - box_w / 2) * w)
            y1 = int((y_center - box_h / 2) * h)
            x2 = int((x_center + box_w / 2) * w)
            y2 = int((y_center + box_h / 2) * h)
            cv2.rectangle(mask, (x1, y1), (x2, y2), color=255, thickness=-1)

    cv2.imwrite(mask_path, mask)

print("🎉 전체 완료: 영상 → 이미지 → 마스크 생성까지 ✅")


🖼️ 마스크 생성 중: 100%|█████████████████████| 417/417 [00:03<00:00, 131.20it/s]

🎉 전체 완료: 영상 → 이미지 → 마스크 생성까지 ✅
